In [18]:
import numpy as np


def loadDataSet():
    dataSet=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
             ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
             ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
             ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
             ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
             ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']] # 切分好的词条
    classVec = [0,1,0,1,0,1] # 类别标签向量，1 代表侮辱性词汇，0 代表非侮辱性词汇
    return dataSet,classVec
dataSet,classVec = loadDataSet()

In [2]:
def createVocabList(dataSet):
    vocabSet = set()
    for i in dataSet:
        vocabSet = vocabSet | set(i)
        vocabList = list(vocabSet)
    return vocabList

vocabList = createVocabList(dataSet)
vocabList

['problems',
 'is',
 'help',
 'cute',
 'posting',
 'buying',
 'take',
 'stupid',
 'quit',
 'flea',
 'please',
 'how',
 'garbage',
 'maybe',
 'stop',
 'my',
 'him',
 'dog',
 'steak',
 'ate',
 'mr',
 'dalmation',
 'to',
 'has',
 'love',
 'worthless',
 'licks',
 'food',
 'park',
 'not',
 'so',
 'I']

In [4]:
# 词汇表向量的转化
def word2vec(vocabList, inputSet):
    vec = [0] * len(vocabList)
    for i in inputSet:
        if i in vocabList:
            vec[vocabList.index(i)] = 1
    return vec


In [5]:
def get_trainMat(dataSet):
    trainMat = []
    vocabList = createVocabList(dataSet)
    for i in dataSet:
        vec = word2vec(vocabList, i)
        trainMat.append(vec)
    return trainMat


In [19]:
def trainNB(trainMat, classVec):
    n = len(trainMat)   # 训练的文档数目
    m = len(trainMat[0])  # 每篇文档的词条数，即词汇表长度
    pAb = sum(classVec) / n   # 文档属于侮辱类的概率
    p0Num = np.ones(m)
    p1Num = np.ones(m)
    p0Denom = 2
    p1Denom = 2
    for i in range(n):
        if classVec[i] == 1:
            p1Num += trainMat[i]
            p1Denom += sum(trainMat[i])
        else:
            p0Num += trainMat[i]
            p0Denom += sum(trainMat[i])
    p1v = np.log(p1Num / p1Denom)
    p0v = np.log(p0Num / p0Denom)
    return p0v, p1v, pAb


In [20]:
from functools import reduce

def classifyNB(vec2classify, p0v, p1v, pAb):
    p1 = sum(vec2classify * p1v) + np.log(pAb)
    p0 = sum(vec2classify * p0v) + np.log(1 - pAb)
    if p1 > p0:
        return 1
    else:
        return 0
    

In [21]:
def testingNB(testVec):
    dataSet,classVec = loadDataSet() #创建实验样本
    vocabList = createVocabList(dataSet) #创建词汇表
    trainMat= get_trainMat(dataSet) #将实验样本向量化
    p0V,p1V,pAb = trainNB(trainMat,classVec) #训练朴素贝叶斯分类器
    thisone = word2vec(vocabList, testVec) #测试样本向量化
    if classifyNB(thisone,p0V,p1V,pAb):
        print(testVec,'属于侮辱类') #执行分类并打印分类结果
    else:
        print(testVec,'属于非侮辱类') #执行分类并打印分类结果
#测试样本1
testVec1 = ['love', 'my', 'dalmation']
testingNB(testVec1)

['love', 'my', 'dalmation'] 属于非侮辱类


In [22]:
testVec2 = ['stupid', 'garbage']
testingNB(testVec2)

['stupid', 'garbage'] 属于侮辱类


由朴素贝叶斯分类器的公式可知，利用贝叶斯分类器对文档进行分类时，要计算多个概率的乘积以获得文档属于某个类别的概率。如果其中有一个概率值为 0，那么最后的成绩也为 0。显然，这样是不合理的，为了降低这种影响，可以将所有词的出现数初始化为 1，并将分母初始化为 2。这种做法就叫做拉普拉斯平滑 (Laplace Smoothing）又被称为加 1 平滑，是比较常用的平滑方法，它就是为了解决 0 概率问题
另外一个遇到的问题就是下溢出，这是由于太多很小的数相乘造成的。我们在计算乘积时，由于大部分因子都很小，所以程序会下溢或者得不到正确答案。为了解决这个问题，对乘积结果取自然对数。通过求对数可以避免下溢出或者浮点数舍入导致的错误。同时，采用自然对数进行处理不会有任何损失